# Продвинутое машинное обучение: Домашнее задание 3

Третье домашнее задание посвящено достаточно простой, но, надеюсь, интересной задаче, в которой потребуется творчески применить методы сэмплирования. Как всегда, любые комментарии, новые идеи и рассуждения на тему категорически приветствуются.

В этом небольшом домашнем задании мы попробуем улучшить метод Шерлока Холмса. Как известно, в рассказе The Adventure of the Dancing Men великий сыщик расшифровал загадочные письмена, которые выглядели примерно так:

![](img/dancing_men.png)

Пользовался он для этого так называемым частотным методом: смотрел, какие буквы чаще встречаются в зашифрованных текстах, и пытался подставить буквы в соответствии с частотной таблицей: E — самая частая и так далее.

В этом задании мы будем разрабатывать более современный и продвинутый вариант такого частотного метода. В качестве корпусов текстов для подсчётов частот можете взять что угодно, но для удобства вот вам “Война и мир” по-русски и по-английски:


## 1. Baseline

Реализуйте базовый частотный метод по Шерлоку Холмсу:

- подсчитайте частоты букв по корпусам (пунктуацию и капитализацию можно просто опустить, а вот пробелы лучше оставить);
- возьмите какие-нибудь тестовые тексты (нужно взять по меньшей мере 2-3 предложения, иначе вряд ли сработает), зашифруйте их посредством случайной перестановки символов;
- расшифруйте их таким частотным методом.

In [1]:
import re
import random
from typing import List, Dict, Callable
from collections import Counter

import tqdm
import numpy as np

RUSSIAN_ALPHABET = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя '
ENGLISH_ALPHABET = 'abcdefghijklmnopqrstuvwxyz '


def read_texts(data_path: str) -> List[str]:
    with open(data_path, "r") as input_stream:
        texts = input_stream.read().splitlines()
    
    return texts


def preprocess_rus(text: str):
    text = re.sub(r'[^а-ё ]+', '', text.lower())
    text = re.sub(' +', ' ', text)
    return text


def preprocess_eng(text: str):
    text = re.sub(r'[^a-z ]+', '', text.lower())
    text = re.sub(' +', ' ', text)
    return text


def preprocess_texts(
    texts: List[str], first_sentence: str = '', last_sentence: str = '', alphabet: str = 'rus'
) -> str:

    if alphabet == 'rus':
        texts = [preprocess_rus(text) for text in texts]
    elif alphabet == 'eng':
        texts = [preprocess_eng(text) for text in texts]
    else:
        raise NotImplementedError()

    preprocessed = [text for text in texts if text]

    if first_sentence:
        index_first = texts.index(first_sentence)
        texts = texts[index_first:]

    if last_sentence:
        index_end = texts.index(last_sentence)
        texts = texts[:index_end]
    
    texts = ' '.join(texts)

    return texts

In [2]:
data_path = "data/AnnaKarenina.txt"
first_sentence = "все счастливые семьи похожи друг на друга каждая несчастливая семья несчастлива посвоему"
last_sentence = "примечания"

texts = read_texts(data_path)
corpus = preprocess_texts(texts, first_sentence, last_sentence, 'rus')

In [3]:
# data_path = "data/WarAndPeace.txt"
# first_sentence = "часть первая"
# last_sentence = ""

# texts = read_texts(data_path)
# corpus = preprocess_texts(texts, first_sentence, last_sentence, 'rus')

In [4]:
# data_path = "data/WarAndPeaceEng.txt"
# first_sentence = "well prince so genoa and lucca are now just family estates of the"
# last_sentence = "end of the project gutenberg ebook of war and peace by leo tolstoy"

# texts = read_texts(data_path)
# corpus = preprocess_texts(texts, first_sentence, last_sentence, 'eng')

In [14]:
def get_text_from_corpus(corpus: str, num_words: int):
    words = corpus.split()
    index_from = random.randint(0, len(words) - num_words)
    sample = " ".join(words[index_from: index_from + num_words])
    
    return sample


def encode_chars(text: str, alphabet_from: str, alphabet_to: str = None):
    if alphabet_to is None:
        alphabet_to = alphabet_from
    
    permutation = np.random.permutation(list(alphabet_from))
    encoding = dict(zip(permutation, alphabet_to))
    encoded_text = ''.join(map(encoding.get, text))
    
    return encoded_text


class CharDecoder:
    def __init__(self, corpus: str):
        self.corpus = corpus
        self.corpus_chars = self.get_sorted_chars(self.corpus)
    
    @staticmethod
    def get_sorted_chars(text: str) -> List[chr]:
        sorted_chars = [item[0] for item in Counter(text).most_common()]
        
        return sorted_chars
    
    def decode(self, encoded_text: str) -> str:       
        text_chars = self.get_sorted_chars(encoded_text)
        inverse_encoding = dict(zip(text_chars, self.corpus_chars))
        decoded_text = ''.join(map(inverse_encoding.get, encoded_text))
        
        return decoded_text

In [15]:
text = get_text_from_corpus(corpus, num_words=100)
text

'тем дружески грубым тоном на который нельзя было сердиться ну проси же скорее сказал облонский морщась от досады просительница штабскапитанша калинина просила о невозможном и бестолковом но степан аркадьич по своему обыкновению усадил ее внимательно не перебивая выслушал ее и дал ей подробный совет к кому и как обратиться и даже бойко и складно своим крупным растянутым красивым и четким почерком написал ей записочку к лицу которое могло ей пособить отпустив штабскапитаншу степан аркадьич взял шляпу и остановился припоминая не забыл ли чего оказалось что он ничего не забыл кроме того что хотел забыть жену ах да он опустил голову'

In [16]:
encoded_text = encode_chars(text, RUSSIAN_ALPHABET)
encoded_text

'ясъёоыцнсжбвёуыцчтъёяэдэъёдеёбэяэытзёдсьгихёчтьэёжсыовягжхёдцёпыэжвёнсёжбэыссёжбеиеьёэчьэджбвзёъэыйежгёэяёоэжеотёпыэжвясьгдвреёюяечжбепвяедюеёбеьвдвдеёпыэжвьеёэёдсмэиъэндэъёвёчсжяэьбэмэъёдэёжяспедёеыбеогвшёпэёжмэсъцёэчтбдэмсдв ёцжеовьёссёмдвъеясьгдэёдсёпсысчвмехёмтжьцюеьёссёвёоеьёсзёпэоыэчдтзёжэмсяёбёбэъцёвёбебёэчыеявягжхёвёоенсёчэзбэёвёжбьеодэёжмэвъёбыцпдтъёыежяхдцятъёбыежвмтъёвёшсябвъёпэшсыбэъёдепвжеьёсзёиепвжэшбцёбёьврцёбэяэыэсёъэуьэёсзёпэжэчвягёэяпцжявмёюяечжбепвяедюцёжяспедёеыбеогвшёмихьёюьхпцёвёэжяедэмвьжхёпывпэъвдехёдсёиечтьёьвёшсуэёэбеиеьэжгёшяэёэдёдвшсуэёдсёиечтьёбыэъсёяэуэёшяэёкэясьёиечтягёнсдцёекёоеёэдёэпцжявьёуэьэмц'

In [17]:
char_decoder = CharDecoder(corpus)
decoded_text = char_decoder.decode(encoded_text)
decoded_text

'снд ьрмшнила жрмуяд сотод те лосоряз тнвыгб уяво инрьасыиб тм кроиа шн илорнн илегев оувотилаз дорэеиы ос ьоиеья кроиаснвытахе йсеуилекасетйе леватате кроиаве о тнпогдоштод а унисовлопод то иснкет ерлеьыач ко ипондм оуялтопнтащ миеьав нн птадеснвыто тн кнрнуапеб пяивмйев нн а ьев нз коьроутяз иопнс л лодм а лел оуресасыиб а ьешн уозло а илвеьто ипоад лрмктяд реисбтмсяд лреиапяд а чнслад кочнрлод текаиев нз гекаиочлм л вахм лосорон дожво нз коиоуасы оскмисап йсеуилекасетйм иснкет ерлеьыач пгбв йвбкм а оисетопавиб кракодатеб тн геуяв ва чнжо олегевоиы чсо от тачнжо тн геуяв лродн сожо чсо юоснв геуясы шнтм ею ье от окмисав жовопм'

In [18]:
def accuracy_score(text1, text2):
    """Процент корректно расшифрованных букв"""
    
    correct = sum([a == b for a, b in zip(text1, text2)])
    score = correct / len(text1)
    
    return score

In [19]:
accuracy_score(text, decoded_text)

0.30551181102362207

## 2. Bigrams

Вряд ли в результате получилась такая уж хорошая расшифровка, разве что если вы брали в качестве тестовых данных целые рассказы. Но и Шерлок Холмс был не так уж прост: после буквы E, которая действительно выделяется частотой, дальше он анализировал уже конкретные слова и пытался угадать, какими они могли бы быть. Я не знаю, как запрограммировать такой интуитивный анализ, так что давайте просто сделаем следующий логический шаг:

- подсчитайте частоты биграмм (т.е. пар последовательных букв) по корпусам;
- проведите тестирование аналогично п.1, но при помощи биграмм


In [20]:
class BigramsDecoder:
    def __init__(self, corpus: str):
        self.corpus = corpus
        self.corpus_bigrams = self.get_sorted_bigrams(self.corpus)
    
    @staticmethod
    def get_bigrams(text: str) -> List[str]:
        bigrams = [text[i: i + 2] for i in range(0, len(text) - 1, 2)]

        return bigrams

    def get_sorted_bigrams(self, text: str) -> List[chr]:
        bigrams = self.get_bigrams(text)
        bigrams = [item[0] for item in Counter(bigrams).most_common()]

        return bigrams
    
    def decode(self, encoded_text: str) -> str:       
        text_bigrams = self.get_sorted_bigrams(encoded_text)
        inverse_encoding = dict(zip(text_bigrams, self.corpus_bigrams))
        decoded_text = ''.join(map(inverse_encoding.get, text_bigrams))
        
        return decoded_text

In [21]:
bigrams_decoder = BigramsDecoder(corpus)
decoded_text = bigrams_decoder.decode(encoded_text)
decoded_text

'о е а и  н с вто п о ия наь стнеално к  онгопокакола тни довл енй вом  боту рора чтьпрослоелн  морваза еоллев ерлиретатеомскк етогилвичтдеант рисеы  удааказбыве затинжеодемит ртиесегдо ласльвсю обче г амоавсяой эмесьэтннимс мукиныивх миылтвей жслмаоеаянуеведтрвыиеамар ясошееерухочааддипебеисзничбоожсвдусаняияужекляизспидоч хокикшиднуддр'

In [22]:
accuracy_score(text, decoded_text)

0.03622047244094488

## 3. MCMC on bigrams

Но и это ещё не всё: биграммы скорее всего тоже далеко не всегда работают. Основная часть задания — в том, как можно их улучшить:

- предложите метод обучения перестановки символов в этом задании, основанный на MCMC-сэмплировании, но по-прежнему работающий на основе статистики биграмм;
- реализуйте и протестируйте его, убедитесь, что результаты улучшились.

In [35]:
class MCMCBiramsDecoder(BigramsDecoder):
    def __init__(self, corpus: str, alphabet_from: str, alphabet_to: str = None):
        super().__init__(corpus)
        
        self.bigrams_freq = self.get_bigrams_freq(corpus)
        self.default_freq =  min(self.bigrams_freq.values())

        if alphabet_to is None:
            alphabet_to = alphabet_from

        self.alphabet_from = list(alphabet_from)
        self.alphabet_to = list(alphabet_to)
    
    def get_bigrams_freq(self, text: str) -> Dict[str, float]:
        bigrams = self.get_bigrams(text)
        counts = dict(Counter(bigrams).most_common())
        
        smoothing_coef = len(text) + len(set(text)) ** 2
        freq = {bigram: count / smoothing_coef for bigram, count in counts.items()}
        
        return freq

    def text_log_proba(self, text: str):
        log_proba = 0
        for i in range(len(text) - 1):
            bigram = text[i: i + 2]
            bigram_freq = self.bigrams_freq.get(bigram, self.default_freq)
            log_proba += np.log(bigram_freq)
        
        return log_proba
    
    
    def decode(self, encoded_text: str, num_trials: int = 10, num_steps: int = 3000) -> str:
        best_log_proba = float("-inf")
        best_inverse_encoding = None
        
        for trial in tqdm.tqdm(range(num_trials)):
            # начнем с прямого отображения
            alphabet = self.alphabet_to.copy()
            inverse_encoding = dict(zip(self.alphabet_from, alphabet))
            decoded_text = ''.join(map(inverse_encoding.get, encoded_text))
            log_proba = self.text_log_proba(decoded_text)
            
            for step in range(num_steps):
                # будем случайным образом менять порядок биграм в корпусе
                new_alphabet = alphabet.copy()
                i, j = np.random.choice(len(new_alphabet), size=2, replace=False)
                new_alphabet[i], new_alphabet[j] = new_alphabet[j], new_alphabet[i]
                
                # считаем логарифм правдоподобия
                new_inverse_encoding = dict(zip(self.alphabet_from, new_alphabet))                
                new_decoded_text = ''.join(map(new_inverse_encoding.get, encoded_text))
                new_log_proba = self.text_log_proba(new_decoded_text)

                # получаем вероятность
                p = np.exp(new_log_proba - log_proba)
                if p > random.random():
                    log_proba = new_log_proba
                    inverse_encoding = new_inverse_encoding
                    decoded_text = new_decoded_text
                    alphabet = new_alphabet
            
            if log_proba > best_log_proba:
                best_log_proba = log_proba
                best_inverse_encoding = inverse_encoding
        
        decoded_text = ''.join(map(best_inverse_encoding.get, encoded_text))
        
        return decoded_text

In [36]:
mcmc_bigram_decoder = MCMCBiramsDecoder(corpus, RUSSIAN_ALPHABET)
decoded_text = mcmc_bigram_decoder.decode(encoded_text)

 40%|████      | 4/10 [00:08<00:13,  2.18s/it]<ipython-input-35-0017bd4f79a6>:56: RuntimeWarning: overflow encountered in exp
  p = np.exp(new_log_proba - log_proba)
100%|██████████| 10/10 [00:21<00:00,  2.17s/it]


In [37]:
decoded_text

'тем дружески грубым тоном на который нельзя было сердиться ну проси же скорее сказал облонский морщась от досады просительница штабскапитанша калинина просила о невозможном и бестолковом но степан аркадьич по своему обыкновению усадил ее внимательно не перебивая выслушал ее и дал ей подробный совет к кому и как обратиться и даже бойко и складно своим крупным растянутым красивым и четким почерком написал ей записочку к лицу которое могло ей пособить отпустив штабскапитаншу степан аркадьич взял шляпу и остановился припоминая не забыл ли чего оказалось что он ничего не забыл кроме того что хотел забыть жену ах да он опустил голову'

In [39]:
accuracy_score(text, decoded_text)

1.0

## 4. Predict

Расшифруйте сообщение:

In [40]:
unknown_text = "←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏"
unknown_alphabet = list(set(unknown_text))

In [43]:
mcmc_bigram_decoder = MCMCBiramsDecoder(corpus, alphabet_from=unknown_alphabet, alphabet_to=RUSSIAN_ALPHABET)
decoded_text = mcmc_bigram_decoder.decode(unknown_text, num_steps=10000)

100%|██████████| 10/10 [00:27<00:00,  2.76s/it]


In [44]:
decoded_text

'если вы вимите нордальный или почти нордальный текст у этого сообщения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный балл за послемнее четвертое замание курса хотя конечно я ничего не обещаж'

## 5*. Trigrams

Бонус: а что если от биграмм перейти к триграммам (тройкам букв) или даже больше? Улучшатся ли результаты? Когда улучшатся, а когда нет? Чтобы ответить на этот вопрос эмпирически, уже может понадобиться погенерировать много тестовых перестановок и последить за метриками, глазами может быть и не видно.

In [ ]:
# TODO

## 6*. Applications
Какие вы можете придумать применения для этой модели? Пляшущие человечки ведь не так часто встречаются в жизни (хотя встречаются! и это самое потрясающее во всей этой истории, но об этом я расскажу потом).

Можно попробовать применить такую зашифровку с последующей расшифровкой в качестве аугментации (добавление шума) в некоторый текстовый корпус. Тогда скорее всего будет меньше вероятность переобучиться на некоторые n-граммы